In [1]:
import pandas as pd
import numpy as np
import re
from glob import glob
import regex


In [2]:
global data_dir, output_dir
data_dir = '../../data/original/external/schmoch/'
output_dir = '../../data/processed/external/schmoch/'


In [3]:
df_dict = pd.read_excel(f'{data_dir}ipc_technology.xlsx', 
                        sheet_name=None, 
                        engine='openpyxl', 
                        header=6, 
                        usecols=['Field_number', 'Sector_en', 'Field_en', 'IPC_code'])
df_dict.keys()

dict_keys(['35 Fields of Technology', 'Energy Technology'])

In [4]:
df_35 = df_dict['35 Fields of Technology'].copy()
df_35['IPC_code'] = df_35['IPC_code'].str.replace('%', '')
df_35


,Field_number,Sector_en,Field_en,IPC_code
0,1,Electrical engineering,"Electrical machinery, apparatus, energy",F21H
1,1,Electrical engineering,"Electrical machinery, apparatus, energy",F21K
2,1,Electrical engineering,"Electrical machinery, apparatus, energy",F21L
3,1,Electrical engineering,"Electrical machinery, apparatus, energy",F21S
4,1,Electrical engineering,"Electrical machinery, apparatus, energy",F21V
...,...,...,...,...
765,35,Other fields,Civil engineering,E21B
766,35,Other fields,Civil engineering,E21C
767,35,Other fields,Civil engineering,E21D
768,35,Other fields,Civil engineering,E21F


In [6]:
df_35.to_csv(f'{output_dir}35technology.csv', 
             sep=',', 
             encoding='utf-8', 
             index=False)


---


In [ ]:
# energyを細かく見る場合
df = pd.concat([value for value in df_dict.values()], 
               axis="index", 
               ignore_index=True)\
    .drop_duplicates(subset=['IPC_code'], keep='first')
df['IPC_code'] = df['IPC_code'].str.replace('%', '').str.replace('#', '')
df['IPC_class'] = df['IPC_code'].str.replace('%', '').str.replace('#', '').str[:3]
df['IPC_subclass'] = df['IPC_code'].str.replace('%', '').str.replace('#', '').str[:4]
df


,Field_number,Sector_en,Field_en,IPC_code,IPC_class,IPC_subclass
0,1,Electrical engineering,"Electrical machinery, apparatus, energy",F21H,F21,F21H
1,1,Electrical engineering,"Electrical machinery, apparatus, energy",F21K,F21,F21K
2,1,Electrical engineering,"Electrical machinery, apparatus, energy",F21L,F21,F21L
3,1,Electrical engineering,"Electrical machinery, apparatus, energy",F21S,F21,F21S
4,1,Electrical engineering,"Electrical machinery, apparatus, energy",F21V,F21,F21V
...,...,...,...,...,...,...
832,94,Energy technology,Hydro energy,B63H 19/04,B63,B63H
833,94,Energy technology,Hydro energy,E02B 9/00,E02,E02B
834,94,Energy technology,Hydro energy,E02B 9/02,E02,E02B
835,94,Energy technology,Hydro energy,E02B 9/04,E02,E02B


In [ ]:
df[(df['IPC_code'] != df['IPC_subclass']) & (df['IPC_code'] != df['IPC_class'])]

,Field_number,Sector_en,Field_en,IPC_code,IPC_class,IPC_subclass
33,2,Electrical engineering,Audio-visual technology,H04N 3/,H04,H04N
34,2,Electrical engineering,Audio-visual technology,H04N 5/,H04,H04N
35,2,Electrical engineering,Audio-visual technology,H04N 7/,H04,H04N
36,2,Electrical engineering,Audio-visual technology,H04N 9/,H04,H04N
37,2,Electrical engineering,Audio-visual technology,H04N 11/,H04,H04N
...,...,...,...,...,...,...
832,94,Energy technology,Hydro energy,B63H 19/04,B63,B63H
833,94,Energy technology,Hydro energy,E02B 9/00,E02,E02B
834,94,Energy technology,Hydro energy,E02B 9/02,E02,E02B
835,94,Energy technology,Hydro energy,E02B 9/04,E02,E02B


In [ ]:
for uni_str in df[(df['IPC_code'] != df['IPC_subclass']) & (df['IPC_code'] != df['IPC_class'])]['IPC_subclass'].unique():
    print('=======================================')
    display(df[df['IPC_code']==uni_str])
    display(df[df['IPC_subclass']==uni_str])

,Field_number,Sector_en,Field_en,IPC_code,IPC_class,IPC_subclass


,Field_number,Sector_en,Field_en,IPC_code,IPC_class,IPC_subclass
33,2,Electrical engineering,Audio-visual technology,H04N 3/,H04,H04N
34,2,Electrical engineering,Audio-visual technology,H04N 5/,H04,H04N
35,2,Electrical engineering,Audio-visual technology,H04N 7/,H04,H04N
36,2,Electrical engineering,Audio-visual technology,H04N 9/,H04,H04N
37,2,Electrical engineering,Audio-visual technology,H04N 11/,H04,H04N
38,2,Electrical engineering,Audio-visual technology,H04N 13/,H04,H04N
39,2,Electrical engineering,Audio-visual technology,H04N 15/,H04,H04N
40,2,Electrical engineering,Audio-visual technology,H04N 17/,H04,H04N
41,2,Electrical engineering,Audio-visual technology,H04N 19/,H04,H04N
42,2,Electrical engineering,Audio-visual technology,H04N 23/,H04,H04N


,Field_number,Sector_en,Field_en,IPC_code,IPC_class,IPC_subclass


,Field_number,Sector_en,Field_en,IPC_code,IPC_class,IPC_subclass
111,10,Instruments,Measurement,G01N 1/,G01,G01N
112,10,Instruments,Measurement,G01N 3/,G01,G01N
113,10,Instruments,Measurement,G01N 5/,G01,G01N
114,10,Instruments,Measurement,G01N 7/,G01,G01N
115,10,Instruments,Measurement,G01N 9/,G01,G01N
116,10,Instruments,Measurement,G01N 11/,G01,G01N
117,10,Instruments,Measurement,G01N 13/,G01,G01N
118,10,Instruments,Measurement,G01N 15/,G01,G01N
119,10,Instruments,Measurement,G01N 17/,G01,G01N
120,10,Instruments,Measurement,G01N 19/,G01,G01N


,Field_number,Sector_en,Field_en,IPC_code,IPC_class,IPC_subclass


...

,Field_number,Sector_en,Field_en,IPC_code,IPC_class,IPC_subclass
628,32,Mechanical engineering,Transport,B60L,B60,B60L
825,92,Energy technology,Wind energy technology,B60L 8/00,B60,B60L


,Field_number,Sector_en,Field_en,IPC_code,IPC_class,IPC_subclass
658,32,Mechanical engineering,Transport,B63H,B63,B63H


,Field_number,Sector_en,Field_en,IPC_code,IPC_class,IPC_subclass
658,32,Mechanical engineering,Transport,B63H,B63,B63H
831,94,Energy technology,Hydro energy,B63H 19/02,B63,B63H
832,94,Energy technology,Hydro energy,B63H 19/04,B63,B63H


,Field_number,Sector_en,Field_en,IPC_code,IPC_class,IPC_subclass
744,35,Other fields,Civil engineering,E02B,E02,E02B


,Field_number,Sector_en,Field_en,IPC_code,IPC_class,IPC_subclass
744,35,Other fields,Civil engineering,E02B,E02,E02B
833,94,Energy technology,Hydro energy,E02B 9/00,E02,E02B
834,94,Energy technology,Hydro energy,E02B 9/02,E02,E02B
835,94,Energy technology,Hydro energy,E02B 9/04,E02,E02B
836,94,Energy technology,Hydro energy,E02B 9/06,E02,E02B
